# Homework Assignment 5 - Evan Callaghan

### 1. If a decision tree is under-fitting the training dataset, is it a good idea to try scaling the input features?

#### If a decision tree is under-fitting the training dataset, it will not make a difference if the input features are scaled or not. Decision trees do not require feature scaling to be performed in order for them to be used in the model.

### 2.  If a decision tree is over-fitting the training dataset, is it a good idea to try decreasing max depth?

#### A large max depth value can lead to excessive complexity in a model, leading to over-fitting the data. Therefore, if a decision tree is over-fitting the training dataset, it is a good idea to decrease the max depth as that will create a more simplistic model.

### 3. Why would you use a random forest instead of a decision tree?

#### D. You would use a random forest instead of a decision tree for a lower training error and to reduce the variance of the model.

### 4. Which of the following is/are TRUE about bagging trees?

#### D. In bagging trees, the trees are grown independent of each other. And bagging is a method for improving the performance by aggregating the results of weak learners.

### 5.  Suppose you are building random forest model which splits a node on the attribute that has highest information gain (using the Gini index). In the below image, which attribute has the highest information gain? Show all your calculations.

#### Outlook Node:
#### Gini_leaf1 = 1 - (2/5)^2 - (3/5)^2 = 12/25
#### Gini_leaf2 = 1 - (4/4)^2 - (0/4)^2 = 0
#### Gini_leaf3 = 1 - (3/5)^2 - (2/5)^2 = 12/25
#### Gini Index = 0.32

#### Temperature Node:
#### Gini_leaf1 = 1 - (2/4)^2 - (2/4)^2 = 0.5
#### Gini_leaf2 = 1 - (4/6)^2 - (2/6)^2 = 0.444
#### Gini_leaf3 = 1 - (3/4)^2 - (1/4)^2 = 0.375
#### Gini Index = 0.4398

#### Windy Node:
#### Gini_leaf1 = 1 - (6/8)^2 - (2/8)^2 = 0.375
#### Gini_leaf2 = 1 - (3/6)^2 - (3/6)^2 = 0.5
#### Gini Index = 0.4375

#### Humidity Node:
#### Gini_leaf1 = 1 - (3/7)^2 - (4/7)^2 = 0.4898
#### Gini_leaf2 = 1 - (6/7)^2 - (1/7)^2 = 0.2449
#### Gini Index = 0.3673

#### Result: The Outlook attribute has the highest information gain because it has the lowest average Gini index. 

In [1]:
## 6. a) Using the pandas library to read the csv data file and create a data-frame called heart

import boto3
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import recall_score
import matplotlib.pyplot as plt

## Defining the bucket
s3 = boto3.resource('s3')
bucket_name = 'data-445-bucket-callaghan'
bucket = s3.Bucket(bucket_name)

## Defining the csv file
file_key = 'framingham.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

heart = pd.read_csv(file_content_stream)

heart.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [2]:
## b)  Removing observations with missing values

heart = heart.dropna()

In [3]:
## c)  Using all the available variables as the predictor variables, and TenYearCHD as the target variable, to do the following:

## (i) Splitting the data into train (80%) and test (20%) (taking into account the proportion of 0s and 1s)
## (ii) Using the train data-frame to build a random forest classifier (using 500 trees).
### (iii) Extract the feature importance of each of the variables.

## Defining the empty data frame to store results
importance = pd.DataFrame(columns = heart.columns)
importance = importance.drop(columns = ['TenYearCHD'])

## Repeating (i)-(iii) 100 times
for i in range(0, 100):

    ## Defining the input and target variables
    X = heart.drop(columns = ['TenYearCHD'])
    Y = heart['TenYearCHD']

    ## Splitting the data
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify = Y)
    
    ## Building the model
    rf_md = RandomForestClassifier(n_estimators = 500).fit(X_train, Y_train)

    ## Extracting the feature importance
    importance.loc[i] = rf_md.feature_importances_.T

    
## Computing the average importance of each of the variables across the 100 splits.
importance.mean()


## sysBP, BMI, age, totChol, and glucose are found to be the five most important variables on average

male               0.021398
age                0.125094
education          0.036859
currentSmoker      0.012567
cigsPerDay         0.050446
BPMeds             0.007056
prevalentStroke    0.003277
prevalentHyp       0.018345
diabetes           0.006627
totChol            0.122062
sysBP              0.134741
diaBP              0.118751
BMI                0.127023
heartRate          0.096058
glucose            0.119696
dtype: float64

In [4]:
## d) Using the top 5 variables from part (c) as the predictor variables and TenYearCHD as the target variable to do the following:

## (i) Splitting the data into train (80%) and test (20%) (taking into account the proportion of 0s and 1s)
## (ii) Using the train data-frame to build a random forest classifier (using 500 trees and maximum depth tree equal to 3). Using this 
## model to predict on the test set
## (iii) Using the train data-frame to build a random forest classifier (using 500 trees and maximum depth tree equal to 5). Using this 
## model to predict on the test set
## (iv) Using the train data-frame to build a random forest classifier (using 500 trees and maximum depth tree equal to 7). Using this 
## model to predict on the test set

## Initalizing empty lists to store results
md1_recall = []
md2_recall = []
md3_recall = []

## Repeating (i)-(iv) 100 times
for i in range(0, 100):

    ## Defining the input and target variables
    X = heart[['sysBP', 'BMI', 'age', 'totChol', 'glucose']]
    Y = heart['TenYearCHD']

    ## Splitting the data
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify = Y)
    
    
    ## Model 1
    ## --------------------------------------------------------

    ## Building the model
    md1 = RandomForestClassifier(n_estimators = 500, max_depth = 3).fit(X_train, Y_train)
    
    ## Predicting on the test set
    md1_preds = md1.predict_proba(X_test)[:, 1]
    
    ## Using 10% as cutoff value and reporting the recall
    md1_preds = np.where(md1_preds < 0.1, 0, 1)
    
    md1_recall.append(recall_score(Y_test, md1_preds))
    
    
    
    ## Model 2
    ## --------------------------------------------------------

    ## Building the model
    md2 = RandomForestClassifier(n_estimators = 500, max_depth = 5).fit(X_train, Y_train)
    
    ## Predicting on the test set
    md2_preds = md2.predict_proba(X_test)[:, 1]
    
    ## Using 10% as cutoff value and reporting the recall
    md2_preds = np.where(md2_preds < 0.1, 0, 1)
    
    md2_recall.append(recall_score(Y_test, md2_preds))
    
    
    
    ## Model 3
    ## --------------------------------------------------------

    ## Building the model
    md3 = RandomForestClassifier(n_estimators = 500, max_depth = 7).fit(X_train, Y_train)
    
    ## Predicting on the test set
    md3_preds = md3.predict_proba(X_test)[:, 1]
    
    ## Using 10% as cutoff value and reporting the recall
    md3_preds = np.where(md3_preds < 0.1, 0, 1)
    
    md3_recall.append(recall_score(Y_test, md3_preds))
    

## Computing the average recall of each of the models across the 100 iterations
print('Average Recall Score Model 1:', np.mean(md1_recall))
print('Average Recall Score Model 2:', np.mean(md2_recall))
print('Average Recall Score Model 3:', np.mean(md3_recall))


## We would use Model 1 (max_depth = 3) to predict TenYearCHD becasue it has the highest recall score of the 
## three considered models.

Average Recall Score Model 1: 0.8360714285714286
Average Recall Score Model 2: 0.8218750000000001
Average Recall Score Model 3: 0.8080357142857143
